In [2]:
import pyspark

In [3]:
pyspark.__file__

'/usr/local/lib/python3.9/site-packages/pyspark/__init__.py'

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/01 13:51:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-01 13:52:55--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.204.145
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.204.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  42.0MB/s    in 17s     

2022-03-01 13:53:12 (41.0 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [32]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

In [33]:
!head fhvhv_tripdata_2021-02.csv

In [34]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [25]:
df.show()


+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [47]:
import pandas as pd

In [48]:
from pyspark.sql import types

In [49]:
fhvhv_schema = types.StructType([
    types.StructField("hvfhs_license_num", types.StringType(), True),
    types.StructField("dispatching_base_num", types.StringType(), True),
    types.StructField("pickup_datetime", types.TimestampType(), True),
    types.StructField("dropoff_datetime", types.TimestampType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("SR_Flag", types.StringType(), True)
])








In [50]:
df = spark.read \
    .option("header", "true") \
    .schema(fhvhv_schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [51]:
df = df.repartition(24)

In [52]:
df.write.parquet('fhvhv/2021/02/')

22/03/01 15:47:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 15:47:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 15:47:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 15:47:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [56]:
!du -h fhvhv

221M	fhvhv/2021/02
221M	fhvhv/2021
221M	fhvhv


In [53]:
df.printSchema()


root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [54]:
from pyspark.sql import functions as F

In [55]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02878|2021-02-02 18:18:35|2021-02-02 18:31:43|          70|          83|   null|
|           HV0005|              B02510|2021-02-04 05:08:43|2021-02-04 05:20:04|         242|         254|   null|
|           HV0003|              B02872|2021-02-03 18:10:19|2021-02-03 18:26:11|         145|         263|   null|
|           HV0003|              B02765|2021-02-05 05:11:52|2021-02-05 05:15:50|          37|          37|   null|
|           HV0005|              B02510|2021-02-02 17:52:04|2021-02-02 18:23:03|         132|          61|   null|
|           HV0003|              B02889|2021-02-02 18:54:44|2021-02-02 19:21:05|

In [58]:
df.registerTempTable('fhvhv')

In [59]:
spark.sql("""
SELECT
    COUNT(1)
FROM
    fhvhv
WHERE
    DATE(pickup_datetime) = '2021-02-15'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [60]:
from pyspark.sql.functions import col, asc,desc

In [66]:
df \
    .withColumn('duration_seconds', df.dropoff_datetime.cast('long')-df.pickup_datetime.cast('long')) \
    .orderBy(col('duration_seconds').desc()) \
    .show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|duration_seconds|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------------+
|           HV0005|              B02510|2021-02-11 13:40:44|2021-02-12 10:39:44|         247|          41|   null|           75540|
|           HV0004|              B02800|2021-02-17 15:54:53|2021-02-18 07:48:34|         242|         254|   null|           57221|
|           HV0004|              B02800|2021-02-20 12:08:15|2021-02-21 00:22:14|         188|          55|   null|           44039|
|           HV0003|              B02864|2021-02-03 20:24:25|2021-02-04 07:41:58|          51|         147|   null|           40653|
|           HV0003|              B02887|2021-02-19 23:17:44|2021-02-20 09:44

In [67]:
spark.sql("""
SELECT
    pickup_datetime, dropoff_datetime,
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) AS duration
FROM
    fhvhv
ORDER BY
    duration DESC
""").show()

+-------------------+-------------------+--------+
|    pickup_datetime|   dropoff_datetime|duration|
+-------------------+-------------------+--------+
|2021-02-11 13:40:44|2021-02-12 10:39:44|   75540|
|2021-02-17 15:54:53|2021-02-18 07:48:34|   57221|
|2021-02-20 12:08:15|2021-02-21 00:22:14|   44039|
|2021-02-03 20:24:25|2021-02-04 07:41:58|   40653|
|2021-02-19 23:17:44|2021-02-20 09:44:01|   37577|
|2021-02-25 17:13:35|2021-02-26 02:57:05|   35010|
|2021-02-20 01:36:13|2021-02-20 11:16:19|   34806|
|2021-02-18 15:24:19|2021-02-19 01:01:11|   34612|
|2021-02-18 01:31:20|2021-02-18 11:07:15|   34555|
|2021-02-10 20:51:39|2021-02-11 06:21:08|   34169|
|2021-02-10 01:56:17|2021-02-10 10:57:33|   32476|
|2021-02-25 09:18:18|2021-02-25 18:18:57|   32439|
|2021-02-21 19:59:13|2021-02-22 04:56:16|   32223|
|2021-02-09 18:36:13|2021-02-10 03:31:00|   32087|
|2021-02-06 09:48:09|2021-02-06 18:32:16|   31447|
|2021-02-02 09:42:30|2021-02-02 18:17:43|   30913|
|2021-02-10 10:12:08|2021-02-10

In [68]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(dispatching_base_num) as maxbasenum
FROM
    fhvhv
GROUP BY
    dispatching_base_num
ORDER BY
    maxbasenum DESC
""").show()

+--------------------+----------+
|dispatching_base_num|maxbasenum|
+--------------------+----------+
|              B02510|   3233664|
|              B02764|    965568|
|              B02872|    882689|
|              B02875|    685390|
|              B02765|    559768|
|              B02869|    429720|
|              B02887|    322331|
|              B02871|    312364|
|              B02864|    311603|
|              B02866|    311089|
|              B02878|    305185|
|              B02682|    303255|
|              B02617|    274510|
|              B02883|    251617|
|              B02884|    244963|
|              B02882|    232173|
|              B02876|    215693|
|              B02879|    210137|
|              B02867|    200530|
|              B02877|    198938|
+--------------------+----------+
only showing top 20 rows



In [69]:
df_zones = spark.read.parquet('zones/')

In [71]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [73]:
zpu = df_zones \
    .withColumnRenamed('Zone', 'PUZone') \
    .withColumnRenamed('LocationID', 'PULocationID') \
    .withColumnRenamed('Borough', 'PUBorough') \
    .drop('service_zone')
zdo = df_zones \
    .withColumnRenamed('Zone', 'DOZone') \
    .withColumnRenamed('LocationID', 'DOLocationID') \
    .withColumnRenamed('Borough', 'DOBorough') \
    .drop('service_zone')


In [74]:
df_join_temp = df.join(zpu, df.PULocationID == zpu.PULocationID)


In [76]:
df_join_temp.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+------------+-------------+--------------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|PULocationID|    PUBorough|              PUZone|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+------------+-------------+--------------------+
|           HV0003|              B02878|2021-02-02 18:18:35|2021-02-02 18:31:43|          70|          83|   null|          70|       Queens|       East Elmhurst|
|           HV0005|              B02510|2021-02-04 05:08:43|2021-02-04 05:20:04|         242|         254|   null|         242|        Bronx|Van Nest/Morris Park|
|           HV0003|              B02872|2021-02-03 18:10:19|2021-02-03 18:26:11|         145|         263|   null|         145|       Queens|Long Island City/...|
|           HV0003|   

In [77]:
df_join = df_join_temp.join(zdo, df_join_temp.DOLocationID == zdo.DOLocationID)

In [78]:
df_join.head()


Row(hvfhs_license_num='HV0003', dispatching_base_num='B02878', pickup_datetime=datetime.datetime(2021, 2, 2, 18, 18, 35), dropoff_datetime=datetime.datetime(2021, 2, 2, 18, 31, 43), PULocationID=70, DOLocationID=83, SR_Flag=None, PULocationID='70', PUBorough='Queens', PUZone='East Elmhurst', DOLocationID='83', DOBorough='Queens', DOZone='Elmhurst/Maspeth')

In [80]:
df_join.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+------------+-------------+--------------------+------------+-------------+--------------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|PULocationID|    PUBorough|              PUZone|DOLocationID|    DOBorough|              DOZone|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+------------+-------------+--------------------+------------+-------------+--------------------+
|           HV0003|              B02878|2021-02-02 18:18:35|2021-02-02 18:31:43|          70|          83|   null|          70|       Queens|       East Elmhurst|          83|       Queens|    Elmhurst/Maspeth|
|           HV0005|              B02510|2021-02-04 05:08:43|2021-02-04 05:20:04|         242|         254|   null|         242|        Bronx|Van Nest/Morris

In [81]:
df_join.drop('PULocationID', 'DOLocationID', 'PULocationID', 'DOLocationID').write.parquet('tmp/hw5')

22/03/01 22:01:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 22:01:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 22:01:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [84]:
df_join = spark.read.parquet('tmp/hw5')

In [85]:
df_join

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, SR_Flag: string, PUBorough: string, PUZone: string, DOBorough: string, DOZone: string]

In [94]:
df_join.registerTempTable('zonestable')

In [95]:
spark.sql("""
SELECT
    CONCAT(coalesce(PUZone, 'Unknown'), '/', coalesce(DOZone, 'Unknown')) AS PDpair,
    COUNT(1)
FROM
    zonestable
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT
    1
;
""").show()

+--------------------+--------+
|              PDpair|count(1)|
+--------------------+--------+
|East New York/Eas...|   45041|
+--------------------+--------+

